**Prompting** is the process of providing a partial, usually text, input to a model. As we discussed in the last chapter, models will then use their parameterized data transformations to find a probable completion or output that matches the prompt.

**Prompt Engineering** is the emerging developer task of designing and optimizing prompts for AI models to achieve specific goals or outcomes. It involves creating high-quality inputs that can elicit accurate and relevant responses from AI models. The next several examples will help get you up to speed on common prompt engineering strategies.

```
               +-------------------+
               |                   |
               |                   |  Completion
Prompt         |       Large       |  Generated text
--------------->     Language      +------------->
               |       Model       |
               |       (LLM)       |
               |                   |
               +-------------------+
```

# Dependencies and imports

In [ ]:
! pip install predictionguard

In [ ]:
import os

import predictionguard as pg
from getpass import getpass

In [ ]:
pg_access_token = getpass('Enter your Prediction Guard access token: ')
os.environ['PREDICTIONGUARD_TOKEN'] = pg_access_token

# Autocomplete

Because LLMs are configured/ trained to perform the task of text completion, the most basic kind of prompt that you might provide is an **autocomplete** prompt. Regardless of prompt structure, the model function will compute the probabilities of words, tokens, or characters that might follow in the sequence of words, tokens, or characters that you provided in the prompt.

Depending on the desired outcome, the prompt may be a single sentence, a paragraph, or even an partial story. Additionally, the prompt may be open-ended, providing only a general topic or theme, or it may be more specific, outlining a particular scenario or plot.

In [ ]:
pg.Completion.create(model="Falcon-7B-Instruct",
    prompt="Daniel Whitenack, a long forgotten wizard from the Lord of the Rings, entered into Mordor to"
)['choices'][0]['text']

In [ ]:
pg.Completion.create(model="Falcon-7B-Instruct",
    prompt="Today I inspected the engine mounting equipment. I found a problem in one of the brackets so"
)['choices'][0]['text']

In [ ]:
pg.Completion.create(model="Falcon-7B-Instruct",
    prompt="""CREATE TABLE llm_queries(id SERIAL PRIMARY KEY, name TEXT NOT NULL, value REAL);
INSERT INTO llm_queries('Daniel Whitenack', 'autocomplete')
SELECT"""
)['choices'][0]['text']

# Zero shot prompts

Autocomplete is a great place to start, but it is only that: a place to start. Throughout this workshop we will be putting on our prompt engineering hats to do some mind blowing things with generative AI. As we continue along that path, there is a general prompt structure that will pop up over and over again:

```
 Prompt:
+------------------------------------------------------------+
|                                                            |
|  +-------------------------------------------------------+ |
|  | ----------------------------------------------------- | | Task Descrip./
|  | ---------------------------------------               | | Instructions
|  +-------------------------------------------------------+ |
|                                                            |
|  +-------------------------------------------------------+ | Current Input/
|  | -------------                                         | | Context
|  +-------------------------------------------------------+ |
|                                                            |
|  +----------------------------------------+                | Output
|  | --------------------------             |                | Indicator
|  +----------------------------------------+                |
|                                                            |
+------------------------------------------------------------+
```

One of the easiest ways to leverage the above prompt structure is to describe a task (e.g., sentiment analysis), provide a single piece of data as context, and then provide a single output indicator. This is called a **zero shot prompt**.

## Sentiment analysis

In [ ]:
pg.Completion.create(model="Falcon-7B-Instruct",
    prompt="""Assign a sentiment label to the text included below. Use the label NEU for neutral sentiment, NEG for negative sentiment, and POS for positive sentiment.

Text: This workshop is spectacular. I love it! So wonderful.
Sentiment:"""
)['choices'][0]['text']

## Question Answering

In [ ]:
prompt = """Read the context below and answer the question. If the question cannot be answered based on the context alone or the context does not explicitly say the answer to the question, write "Sorry I had trouble answering this question, based on the information I found."

Context: Domino's gift cards are great for any person and any occasion. There are a number of different options to choose from. Each comes with a personalized card carrier and is delivered via US Mail.

Question: How are gift cards delivered?

Answer: """

In [ ]:
pg.Completion.create(model="Falcon-7B-Instruct",
    prompt=prompt
)['choices'][0]['text']

## Fraud Detection

In [ ]:
prompt = "Assign a fraud score to a customer of a life insurance company. "
prompt = prompt + "The score ranges from 0 to 1. Scores close to zero represent that the customer is very unlikely to be committing fraud. Scores close to 1 represent that a customer is very likely to be committing fraud. "
prompt = prompt + "Transaction history data includes transaction dates, category, type of insurance product, reason code, reward amount, coverage limit, and income at the time of the transaction.\n\n"
prompt = prompt + """Transaction History:
1. Date: 9/10/1977; Category: IN; Type: T; Reason Code: NaN; Reward Amount: NaN; Coverage Limit: 50000.0; Income: 61000.0
2. Date: 12/31/2005; Category: CL; Type: T; Reason Code: NaN; Reward Amount: NaN; Coverage Limit: 50000.0; Income: 61000.0
3. Date: 1/17/2006; Category: RE; Type: T; Reason Code: 265.0; Reward Amount: 50000.0; Coverage Limit: NaN; Income: NaN
4. Date: 12/15/1998; Category: IN; Type: T; Reason Code: NaN; Reward Amount: NaN; Coverage Limit: 100000.0; Income: 29000.0
5. Date: 6/7/1961; Category: IN; Type: V; Reason Code: NaN; Reward Amount: NaN; Coverage Limit: 100000.0; Income: 48000.0\n\n"""
prompt = prompt + "Fraud score: "

print(prompt)

In [ ]:
pg.Completion.create(model="Falcon-7B-Instruct",
    prompt=prompt
)['choices'][0]['text']

In [ ]:
prompt = "Assign a fraud score to a customer of a life insurance company. "
prompt = prompt + "The score ranges from 0 to 1. Scores close to zero represent that the customer is very unlikely to be committing fraud. Scores close to 1 represent that a customer is very likely to be committing fraud. "
prompt = prompt + "Transaction history data includes transaction dates, category, type of insurance product, reason code, reward amount, coverage limit, and income at the time of the transaction.\n\n"
prompt = prompt + """Statistics of all customer transactions:
Time between claim and reward: average is 23 days, standard deviation is 77 days
Coverage to income ratio: average is 7.2, standard deviation is 4.2
Recency according to an exponential distribution: average is 0.42, standard deviation is 0.24
Reward amount: average is $359,556, standard deviation $260,000\n\n"""
prompt = prompt + """Customer transaction history:
1. Date: 9/10/1977; Category: IN; Type: T; Reason Code: NaN; Reward Amount: NaN; Coverage Limit: 50000.0; Income: 61000.0
2. Date: 12/31/2005; Category: CL; Type: T; Reason Code: NaN; Reward Amount: NaN; Coverage Limit: 50000.0; Income: 61000.0
3. Date: 1/17/2006; Category: RE; Type: T; Reason Code: 265.0; Reward Amount: 50000.0; Coverage Limit: NaN; Income: NaN
4. Date: 12/15/1998; Category: IN; Type: T; Reason Code: NaN; Reward Amount: NaN; Coverage Limit: 100000.0; Income: 29000.0
5. Date: 6/7/1961; Category: IN; Type: V; Reason Code: NaN; Reward Amount: NaN; Coverage Limit: 100000.0; Income: 48000.0\n\n"""
prompt = prompt + "Fraud score: "

print(prompt)

In [ ]:
pg.Completion.create(model="Falcon-7B-Instruct",
    prompt=prompt
)['choices'][0]['text']

# Few Shot prompts

When your task is slightly more complicated or requires a few more leaps in reasoning to generate an appropriate response, you can turn to **few shot** prompting (aka **in context learning**). In few shot prompting, a small number of gold standard demonstrations are integrated into the prompt. These demonstrations serve as example (context, output) pairs for the model, which serve to tune the probable output on-the-fly to what we ideally want in the output.

Although not always necessary (as seen above), few shot prompting generally produces better results than single shot prompting in terms of consistency and similarity to your ideal outputs. This does come at a cost for some models that might charge based on the number of characters or words that are input to the model API.

## Sentiment

In [ ]:
prompt = """Classify the sentiment of the text. Use the label NEU for neutral sentiment, NEG for negative sentiment, and POS for positive sentiment.

Text: That pilot is adorable.
Sentiment: POS

Text: This was an awful seat.
Sentiment: NEG

Text: This pilot was brilliant.
Sentiment: POS

Text: I saw the aircraft.
Sentiment: NEU

Text: That food was exceptional.
Sentiment: POS

Text: That was a private aircraft.
Sentiment: NEU

Text: This is an unhappy pilot.
Sentiment: NEG

Text: The staff is rough.
Sentiment: NEG

Text: This staff is Australian.
Sentiment: NEU

Text: The flight is boring.
Sentiment:"""

In [ ]:
pg.Completion.create(model="Falcon-7B-Instruct",
    prompt=prompt
)['choices'][0]['text']

## Logic representations

In [ ]:
openai_access_token = getpass('Enter your OpenAI API key: ')
os.environ['OPENAI_API_KEY'] = openai_access_token

In [ ]:
prompt = """Transform text statements into first order logic.

Statement: Pluto is a dog
Logic: (dog(Pluto))

Statement: Deb is not tall
Logic: (¬tall(Deb))

Statement: Not every person who plays football knows cricket
Logic: (¬∀(x)(((person(x))∧(plays(x,football)))→(knows(x,cricket))))

Statement: Some people can sell any product
Logic: ∃(x)((people(x))∧∀(y)((product(y))→(sell(x,y))))

Statement: If you enroll in the course and prepare hard then you will pass the course
Logic: (((enroll(you,course))∧(prepare-hard(you)))→(pass(you,course)))

Statement: Do not forget to provide all of your demonstrations
Logic:"""

In [ ]:
pg.Completion.create(model="OpenAI-text-davinci-003",
    prompt=prompt
)['choices'][0]['text']

## Hinglish conversation

In [ ]:
prompt = """The following is a conversation with an AI assistant in Hinglish. The assistant is helpful, creative, clever, and very friendly. The AI assistant responds in Hinglish.

Human: Hello
AI: Hello!

Human: Movie ke baare mein tumhara initial opinion kya hai? Zyada depth mein gaye bina? Tumne like kiya ki nahin?
AI: Overall, mujhe movie achchi lagi.  Usme kuh high-profile actors hain,  Tum kya kahte ho?

Human: Tumhe lagta hai ki ek achche movie mein high-profile actors hone chahiye? Ya phir, ek movie ko achcha kahne ke liye high-profile actors hone chahiye?
AI: Zaroori nahin, par help zaroor milta hai.  Ek actor jab tak apna kaam theek se nahin karta, tab tak high-profile nahin ban sakta. Tumhe kya lagta hai?

Human: muje lgta hai ki yeh kewal personal pasand ke bare mai hai meri pasandida american comedy films Will Ferrol or Seth Rogan ke sath kuch bhi samil slapstick films hai muje yakin hai ki ek film thi jisme Lindsay Lohan ko bhot bada bna diya tha bhale hi usne apne surati saalo mai movies mai acting ki thi
AI: """

In [ ]:
output = pg.Completion.create(model="OpenAI-text-davinci-003",
    prompt=prompt
)['choices'][0]['text']
output

In [ ]:
prompt = """Translate the following text from Hinglish to English.

Text: {hinglish}
Translation: """.format(hinglish=output)

pg.Completion.create(model="OpenAI-text-davinci-003",
    prompt=prompt
)['choices'][0]['text']